In [120]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.core.display import display, HTML
import math
import numpy as np
import pickle

In [3]:
first_html = 'https://www.realtor.com/realestateandhomes-detail/6545-SE-Yamhill-Ct_Portland_OR_97215_M12879-48012'
response = requests.get(first_html, headers={'User-Agent': 'Mozilla/5.0'})
page = response.text
soup = BeautifulSoup(page, "html5lib")

In [4]:
response.status_code

200

In [181]:
#Get all zipcodes for the greater Portland areaz

first_html = 'https://namecensus.com/igapo/zip_codes/metropolitan-areas/metro-zip/Portland%20(OR-WA)1.html'
response = requests.get(first_html, headers={'User-Agent': 'Mozilla/5.0'})
page = response.text
soup = BeautifulSoup(page, "html5lib")
table = soup.find('table')
df_zip = pd.read_html(page)
df_zip[0][0]
zip_code_list = []
for row in df_zip[0][0]:
    zip_code = row.split()[0]
#     print(zip_code)
    zip_code_list.append(zip_code + '/filter/include=sold-3mo')
greaterpdx_zipcode_list_3mo = list(set(zip_code_list))

In [184]:
greaterpdx_zipcode_list_3mo

['97080/filter/include=sold-3mo',
 '97202/filter/include=sold-3mo',
 '97212/filter/include=sold-3mo',
 '97240/filter/include=sold-3mo',
 '97051/filter/include=sold-3mo',
 '97239/filter/include=sold-3mo',
 '97119/filter/include=sold-3mo',
 '97378/filter/include=sold-3mo',
 '97015/filter/include=sold-3mo',
 '98686/filter/include=sold-3mo',
 '97132/filter/include=sold-3mo',
 '97216/filter/include=sold-3mo',
 '97215/filter/include=sold-3mo',
 '97068/filter/include=sold-3mo',
 '97036/filter/include=sold-3mo',
 '97225/filter/include=sold-3mo',
 '97016/filter/include=sold-3mo',
 '97238/filter/include=sold-3mo',
 '97008/filter/include=sold-3mo',
 '98601/filter/include=sold-3mo',
 '97060/filter/include=sold-3mo',
 '97201/filter/include=sold-3mo',
 '97299/filter/include=sold-3mo',
 '97054/filter/include=sold-3mo',
 '97109/filter/include=sold-3mo',
 '97067/filter/include=sold-3mo',
 '97116/filter/include=sold-3mo',
 '97256/filter/include=sold-3mo',
 '98642/filter/include=sold-3mo',
 '97023/filter

In [169]:
def pdx_only_zipcodes(zipcode1, zipcode2):
    url_list = []
    for zip_code in range(zipcode1, zipcode2 + 1):    
        url = str(zip_code) + "/filter/include=sold-3mo"
        url_list.append(url)
    return url_list

pdx_zipcode_list_3mo = pdx_only_zipcodes(97201, 97266)

In [170]:
def parse_house_links(parsed_html):
    direct_house_links = []
    all_links_on_page = parsed_html.find_all(class_ = "slider-item")
    for link in all_links_on_page:
        house_link_text = link['href']
        direct_house_links.append(house_link_text)
        direct_house_links = list(set(direct_house_links))
    return direct_house_links
parse_house_links(test_soup)

['/OR/Portland/911-NE-68th-Ave-97213/home/26448468',
 '/OR/Portland/5405-NE-Everett-St-97213/home/26296386',
 '/OR/Portland/3234-NE-55th-Ave-97213/home/26668871',
 '/OR/Portland/1525-NE-47th-Ave-97213/home/26646064',
 '/OR/Portland/1314-NE-72nd-Ave-97213/home/26507675',
 '/OR/Portland/837-NE-78th-Ave-97213/home/26585415',
 '/OR/Portland/3144-NE-53rd-Ave-97213/unit-A/home/26468592',
 '/OR/Portland/2441-NE-51st-Ave-97213/home/26600432',
 '/OR/Portland/3235-NE-45th-Ave-97213/home/26438929',
 '/OR/Portland/4155-NE-Alameda-St-97212/home/26438058',
 '/OR/Portland/4334-NE-Laurelhurst-Pl-97213/home/25850815',
 '/OR/Portland/4914-NE-Halsey-St-97213/home/26484427',
 '/OR/Portland/405-NE-62nd-Ave-97213/home/26606562',
 '/OR/Portland/3536-NE-75th-Ave-97213/home/26600746',
 '/OR/Portland/202-NE-50th-Ave-97213/home/26595433',
 '/OR/Portland/924-NE-76th-Ave-97213/home/26444359',
 '/OR/Portland/6609-NE-Beech-St-97213/home/26508615',
 '/OR/Portland/227-NE-61st-Ave-97213/unit-2/home/26649844',
 '/OR/Por

In [191]:
# Get links for sold homes in all zipcodes, put in error handling
def get_links(zipcode_list):
    house_html_list = []
    num_search_pages = []
    base_url = "https://www.redfin.com/zipcode/"
    
    #Start up selenium
    chromedriver = '/Applications/chromedriver'
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)   
    for zipcode in zipcode_list:
        zipcode_url = base_url + zipcode
        driver.get(zipcode_url)
        test_soup = BeautifulSoup(driver.page_source, 'html5lib')
        try: 
            num_homes = int(test_soup.find(class_= "homes summary").text.split()[3])
            print(num_homes)
            house_html_list.extend(parse_house_links(test_soup))

            if num_homes > 41:
                num_pages = math.ceil((num_homes/41))
#                 print(num_pages)
                other_pages = [zipcode + '/page-' + str(i) for i in range(2, num_pages+1)]
#                 print(other_pages)
                for other_page in other_pages:
                    zipcode_url = base_url + other_page
                    driver.get(zipcode_url)
                    test_soup = BeautifulSoup(driver.page_source, 'html.parser') 
                    house_html_list.extend(parse_house_links(test_soup))
                    time.sleep(4+2*np.random.rand())
        except:
            time.sleep(4+2*np.random.rand())
            pass
                
    return house_html_list
pdx_only_3mo_links = get_links(pdx_zipcode_list_3mo)
greaterpdx_3mo_links = get_links(greaterpdx_zipcode_list_3mo)

87
3
['97201/filter/include=sold-3mo/page-2', '97201/filter/include=sold-3mo/page-3']
200
5
['97202/filter/include=sold-3mo/page-2', '97202/filter/include=sold-3mo/page-3', '97202/filter/include=sold-3mo/page-4', '97202/filter/include=sold-3mo/page-5']
168
5
['97203/filter/include=sold-3mo/page-2', '97203/filter/include=sold-3mo/page-3', '97203/filter/include=sold-3mo/page-4', '97203/filter/include=sold-3mo/page-5']
296
8
['97206/filter/include=sold-3mo/page-2', '97206/filter/include=sold-3mo/page-3', '97206/filter/include=sold-3mo/page-4', '97206/filter/include=sold-3mo/page-5', '97206/filter/include=sold-3mo/page-6', '97206/filter/include=sold-3mo/page-7', '97206/filter/include=sold-3mo/page-8']
105
3
['97209/filter/include=sold-3mo/page-2', '97209/filter/include=sold-3mo/page-3']
83
3
['97210/filter/include=sold-3mo/page-2', '97210/filter/include=sold-3mo/page-3']
213
6
['97211/filter/include=sold-3mo/page-2', '97211/filter/include=sold-3mo/page-3', '97211/filter/include=sold-3mo/pa

183
5
['97224/filter/include=sold-3mo/page-2', '97224/filter/include=sold-3mo/page-3', '97224/filter/include=sold-3mo/page-4', '97224/filter/include=sold-3mo/page-5']
186
5
['98661/filter/include=sold-3mo/page-2', '98661/filter/include=sold-3mo/page-3', '98661/filter/include=sold-3mo/page-4', '98661/filter/include=sold-3mo/page-5']
98
3
['97214/filter/include=sold-3mo/page-2', '97214/filter/include=sold-3mo/page-3']
239
6
['97219/filter/include=sold-3mo/page-2', '97219/filter/include=sold-3mo/page-3', '97219/filter/include=sold-3mo/page-4', '97219/filter/include=sold-3mo/page-5', '97219/filter/include=sold-3mo/page-6']
243
6
['97217/filter/include=sold-3mo/page-2', '97217/filter/include=sold-3mo/page-3', '97217/filter/include=sold-3mo/page-4', '97217/filter/include=sold-3mo/page-5', '97217/filter/include=sold-3mo/page-6']
134
4
['97269/filter/include=sold-3mo/page-2', '97269/filter/include=sold-3mo/page-3', '97269/filter/include=sold-3mo/page-4']
48
2
['97232/filter/include=sold-3mo/pa

In [195]:
#Pickle the lists!
print(len(pdx_only_3mo_links))
with open('pdx_only_3mo_links.pickle', 'wb') as f:
    pickle.dump(pdx_only_3mo_links, f)
pdx_only_3mo_links[:10]

5874


['/OR/Portland/2820-SW-Fern-St-97201/home/26423002',
 '/OR/Portland/111-SW-Harrison-St-97201/unit-8C/home/25770215',
 '/OR/Portland/111-SW-Harrison-St-97201/unit-7G/home/26447827',
 '/OR/Portland/2809-S-Water-Ave-97201/home/26296031',
 '/OR/Portland/255-SW-Harrison-St-97201/unit-5C/home/25784883',
 '/OR/Portland/2622-SW-Davenport-Ct-97201/home/26575142',
 '/OR/Portland/1510-SW-Hall-St-97201/home/25855291',
 '/OR/Portland/1900-S-River-Dr-97201/unit-N401/home/26625060',
 '/OR/Portland/2730-SW-English-Ct-97201/home/26504705',
 '/OR/Portland/2521-SW-Hoffman-Ave-97201/home/26501343']

In [ ]:
def redfin_address(soup):
    headers = ['streets', 'city', 'region', 'zipcode']
    try:
        street = soup.find(itemprop="streetAddress")[0].text
        city = soup.find_all(itemprop="addressLocality")[0].text
        region = soup.find(itemprop="addressRegion")[0].text
        zipcode = soup.find(itemprop="postalCode")[0].text
        address_dict = 
    except:
        pass
    return 

In [ ]:
def get_house_dict(links):
    base_url = 
    url = base_url + link
    
    #Request HTML 
    response = requests.get(url)
        if response.status_code != 200:
            return None
        page = response.text
        soup = BeautifulSoup(page,"lxml")
    
    #Get address
        street = soup.find(itemprop="streetAddress")[0].text
        city = soup.find_all(itemprop="addressLocality")[0].text
        region = soup.find(itemprop="addressRegion")[0].text
        zipcode = soup.find(itemprop="postalCode")[0].text

    #Create house dictionary and return
        house_dict = dict(zip(headers, [street, city, region, zipcode])
        
        return house_dict

# sold_house_dict_list = 
# for link in sold_house.link_stub:
#     sold_house_dict_list.append(get_house_dict(link))


In [163]:
headers = ['street', 'city', 'region', 'zip_Code']
address_dict = {}
address_soup = soup.find_all(itemprop="streetAddress")[0]
address = address_soup.text
city_soup = soup.find_all(itemprop="addressLocality")[0]
city = city_soup.text
region = region_soup.text
zip_soup = soup.find_all(itemprop="postalCode")[0]
zipcode = zip_soup.text

In [169]:
headers_list = ['Beds', 'Bathrooms', 'Square Feet', 'Lot Size']
gen_info_soup = soup.find_all(class_="data-value")
beds = int(gen_info_soup[0].text)
bathrooms = float(gen_info_soup[1].text.replace('+', ''))
sq_ft = int(gen_info_soup[2].text.replace(',',''))
sqft_lot = int(gen_info_soup[3].text.replace(',',''))
history_soup = 
year_built = 
year_renovated = 


(4, 3.5)

In [178]:
tables = pd.read_html(page)
price_history = tables[1]
listed_price = price_history[price_history['Event'] == 'Listed'].reset_index()
listed_price.iloc(0)

In [172]:
tables[0]
tables[0]['Rating']

,Rating*,School Name,Grades,Distance
0,8,Glencoe Elementary School,K–5,0.7 mi
1,4,Mt Tabor Middle School,6–8,0.5 mi
2,5,Franklin High School,9–12,1.1 mi
3,9,Portland Arthur Academy Charter School,K–5,0.5 mi
4,4,Bridger Elementary School,K–8,0.8 mi
5,4,Madison High School,9–12,1.9 mi
6,NR,Gately Academy Private School,4–8,0.9 mi
7,NR,The Portland Montessori Private School,PK–5,1.0 mi


In [102]:
address = soup.find_all(id = 'ds-chip-property-address')
for element in address:
    print(element.prettify())
street = address[0].text.replace('xa0','')
street

<h1 class="Text-c11n-8-33-0__aiai24-0 StyledHeading-c11n-8-33-0__ktujwe-0 lfNwrZ" font-family="sansSerif" id="ds-chip-property-address">
 <span>
  1527 NW Miller Rd,
 </span>
 <span>
  <!-- -->
  Portland, OR 97229
 </span>
</h1>


'1527 NW Miller Rd,\xa0Portland, OR 97229'

In [131]:
\

Bedrooms: 4
